In [17]:
import geopandas as gpd
import pandas as pd
import dbfread as dbf

In [50]:
def ag_count(state):
    data = gpd.read_file("Data\\SoVI2010_" + state + "\\SoVI0610_" + state + ".shp")
    data['CTFIPS'] = data['STATEFP10'] + data['COUNTYFP10']
    return data.groupby('CTFIPS').agg({"SOVI0610" + state: "mean", "P0010001": "sum"})

CA = ag_count("CA")
FL = ag_count("FL")
IN = ag_count("IN")
LA = ag_count("LA")
ME = ag_count("ME")
WA = ag_count("WA")

,P0010001,SOVI0610FL
CTFIPS,,
12001,247336.0,-0.819564
12003,27115.0,-1.538505
12005,168852.0,-0.786893
12007,28520.0,-1.118205
12009,543369.0,-0.345040
